In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
data = pd.read_csv('news.csv')
data = data.dropna(subset=['rubric'])
# Некоторые тексты не помечены никакой категорией. Никакой пользы они нам не принесут

In [3]:
for c in data['rubric'].unique():
    print(c, sum(data['rubric'] == c))

Экономика 420
Спорт 494
Из жизни 197
Интернет и СМИ 211
Культура 206
Дом 123
Бывший СССР 394
69-я параллель 27
Мир 718
Наука и техника 208
Путешествия 183
Россия 908
Нацпроекты 53
Силовые структуры 220
Ценности 141


In [4]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(np.array(data['rubric']).reshape(-1, 1)).astype('int').toarray()
column_to_category = list(ohe.categories_[0])
# Натренируем 15 классификаторов, каждый по своей категории будет определять, принадлежит кли к ней текст или нет

In [5]:
text = np.array(data['text'])

stem_rus = RussianStemmer(False)

for i in range(len(text)):
    out = ''
    for word in re.sub('[^{0-9а-яА-Я}]', ' ', text[i]).lower().split(' '):
        out += stem_rus.stem(word) + ' '
    text[i] = out
# Лемматизация, конечно, лучше, но для наших целей подойдёт и стемминг

In [6]:
basic_vectorizer = CountVectorizer(min_df=4)
text_train = text[:4000]
text_test = text[4000:]
X_train = basic_vectorizer.fit_transform(text_train)
X_test = basic_vectorizer.transform(text_test)
y_train = y[:4000, -1]
y_test = y[4000:, -1]
# В исходном датасете статьи были перемешаны, поэтому можно разделить даже так

In [7]:
from ensembles import RandomForestMSE

In [8]:
from sklearn.metrics import mean_squared_error

grid = {
    'n_estimators': [30, 50, 100],
    'max_depth': [10, 11]
}

gs_gbmse = GridSearchCV(estimator=RandomForestMSE(n_estimators=10), param_grid=grid, 
    scoring='neg_mean_squared_error', n_jobs=-1)
# GradientBoostingMSE - не библиотечный класс, но он удовлетворяет scikit API
gs_gbmse.fit(X_train, y_train)
gs_gbmse.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [9]:
predictor = RandomForestMSE(n_estimators=500, max_depth=21)

predictor.fit(X_train, y_train)

mean_squared_error(predictor.predict(X_test), y_test), mean_squared_error(np.ones_like(y_test) * np.mean(y_train), y_test)

(0.03335996249102607, 0.08147792929920478)

In [10]:
mean_squared_error(predictor.predict(X_test), y_test, squared=False), mean_squared_error(np.ones_like(y_test) * np.mean(y_train), y_test, squared=False)

(0.18264709822777384, 0.285443390708569)

Точность выше, чем у лучшего константного прогноза, что хорошо

In [11]:
for a, b in zip(predictor.predict(X_test[100:]), y_test[100:]):
    print(a, b)

0.015881799215083935 0
0.015881799215083935 0
0.01605780239532026 0
0.12034149164297553 0
0.015881799215083935 0
0.02596507695122448 0
0.12338936288904048 0
0.017327200369288122 0
0.07824712744294143 0
0.06343736053545858 0
0.015881799215083935 0
0.0187092043461039 0
0.015881799215083935 0
0.015881799215083935 0
0.015881799215083935 0
0.015881799215083935 0
0.015881799215083935 0
0.02568747820868301 0
0.015881799215083935 0
0.8766559030291872 1
0.03852312802773366 0
0.015881799215083935 0
0.059376066534110324 0
0.015881799215083935 0
0.0956977840800555 0
0.01781590364550076 0
0.015881799215083935 0
0.01784693954004702 0
0.24633024452846639 0
0.032060315046462576 0
0.13712765397478238 0
0.058162139421210374 1
0.015881799215083935 0
0.032060315046462576 0
0.015853451808570066 0
0.015881799215083935 0
0.015881799215083935 0
0.016043946837722547 0
0.07742356333389992 0
0.2657292403962293 0
0.016043946837722547 0
0.01691275857438353 0
0.015881799215083935 0
0.015881799215083935 0
0.03021712

In [12]:
from scipy import sparse

X = sparse.csr_matrix(np.vstack((X_train.toarray(), X_test.toarray())))
n_topics = len(column_to_category)
column_to_category, n_topics

(['69-я параллель',
  'Бывший СССР',
  'Дом',
  'Из жизни',
  'Интернет и СМИ',
  'Культура',
  'Мир',
  'Наука и техника',
  'Нацпроекты',
  'Путешествия',
  'Россия',
  'Силовые структуры',
  'Спорт',
  'Ценности',
  'Экономика'],
 15)

In [13]:
list_of_models: list = [None] * n_topics


for i in range(n_topics):
    gb = RandomForestMSE(n_estimators=5000, max_depth=20)
    gb.fit(X, y[:, i])
    list_of_models[i] = gb
    print(i + 1, '/', n_topics)

# Осторожно! Работает несколько часов

1 / 15
2 / 15
3 / 15
4 / 15
5 / 15
6 / 15
7 / 15
8 / 15
9 / 15
10 / 15
11 / 15
12 / 15
13 / 15
14 / 15
15 / 15


In [14]:
list_of_small_models: list = [None] * n_topics

for i in range(n_topics):
    gb = RandomForestMSE(n_estimators=100, max_depth=25)
    gb.fit(X, y[:, i])
    list_of_small_models[i] = gb
    print(i + 1, '/', n_topics)

# Более легковесная модель

1 / 15
2 / 15
3 / 15
4 / 15
5 / 15
6 / 15
7 / 15
8 / 15
9 / 15
10 / 15
11 / 15
12 / 15
13 / 15
14 / 15
15 / 15


In [15]:
list(zip(list_of_small_models[2].predict(X_test[:20]), y[4000:4020, 2]))

[(0.009062727415948588, 0),
 (0.006136853645664303, 0),
 (0.9604040475658792, 1),
 (0.006264582886421783, 0),
 (0.006136853645664303, 0),
 (0.007737415092232405, 0),
 (0.006136853645664303, 0),
 (0.007737415092232405, 0),
 (0.006042427114112022, 0),
 (0.006136853645664303, 0),
 (0.006136853645664303, 0),
 (0.006136853645664303, 0),
 (0.006136853645664303, 0),
 (0.6963425644470395, 1),
 (0.006136853645664303, 0),
 (0.006136853645664303, 0),
 (0.6053245129887859, 1),
 (0.006136853645664303, 0),
 (0.006136853645664303, 0),
 (0.006136853645664303, 0)]

In [16]:
file_model = open('model.pkl', 'wb')
pickle.dump(
    (basic_vectorizer, # преобразователь текста в вектор
    column_to_category, # преобразователь номера лучшей модели в тему
    list_of_models), # список моделей
    file_model)
# сохраняем модель

In [17]:
file_model_small = open('model_small.pkl', 'wb')
pickle.dump(
    (basic_vectorizer, # преобразователь текста в вектор
    column_to_category, # преобразователь номера лучшей модели в тему
    list_of_small_models), # список моделей
    file_model_small)
# сохраняем облегчённую модель, которая помещается в 50мб

In [20]:
from typing import List

file = open('model_small.pkl', 'rb')
data = pickle.load(file)

vectorizer, list_of_topics, list_of_models = data
stem_rus = RussianStemmer(False)

def guess(text: str):
    stem_rus = RussianStemmer(False)

    out = ''
    for word in re.sub('[^{0-9а-яА-Я}]', ' ', text).lower().split(' '):
        out += stem_rus.stem(word) + ' '

    X_from_user = vectorizer.transform([out]).toarray()
    list_of_predictions = []

    for gb in list_of_models:
        list_of_predictions.append(gb.predict(X_from_user))

    top = sorted(list(zip(list_of_predictions, list_of_topics)))[::-1][:3]
    print(top)

In [21]:
with open('texts/economics.txt', encoding='utf-8') as f:
    text = f.read()
    guess(text)

with open('texts/culture.txt', encoding='utf-8') as f:
    text = f.read()
    guess(text)

with open('texts/internet.txt', encoding='utf-8') as f:
    text = f.read()
    guess(text)
    
with open('texts/science.txt', encoding='utf-8') as f:
    text = f.read()
    guess(text)

with open('texts/sport.txt', encoding='utf-8') as f:
    text = f.read()
    guess(text)

with open('texts/travel.txt', encoding='utf-8') as f:
    text = f.read()
    guess(text)

[(array([0.85081633]), 'Экономика'), (array([0.26450042]), 'Дом'), (array([0.11364941]), 'Россия')]
[(array([0.61176337]), 'Культура'), (array([0.12989312]), 'Россия'), (array([0.03962774]), 'Мир')]
[(array([0.52536483]), 'Интернет и СМИ'), (array([0.32340097]), 'Наука и техника'), (array([0.27526893]), 'Мир')]
[(array([0.61635901]), 'Экономика'), (array([0.6113534]), 'Россия'), (array([0.54046375]), 'Наука и техника')]
[(array([0.49]), 'Спорт'), (array([0.44255986]), 'Путешествия'), (array([0.30782726]), 'Интернет и СМИ')]
[(array([0.76029683]), 'Путешествия'), (array([0.10981624]), 'Из жизни'), (array([0.10397408]), 'Спорт')]


Тестирование показало, что модель адекватно справляется с угадыванием тематики статьи. Можно писать сервер. в сервере будем использовать более лёгкую модель, потому что "тяжёлая" думает больше минуты, а "лёгкая" всё-таки способна давать неплохие прогнозы (например, она угадала 5 из 6 тем на статьях, которые видит впервые, а для шестой статьи правильный ответ попал в её топ-3).

Далее из спортивного интереса приведены результаты для "тяжёлой" модели.

In [22]:
from typing import List

file = open('model.pkl', 'rb')
data = pickle.load(file)

vectorizer, list_of_topics, list_of_models = data
stem_rus = RussianStemmer(False)

def guess_long(text: str):
    stem_rus = RussianStemmer(False)

    out = ''
    for word in re.sub('[^{0-9а-яА-Я}]', ' ', text).lower().split(' '):
        out += stem_rus.stem(word) + ' '

    X_from_user = vectorizer.transform([out]).toarray()
    list_of_predictions = []

    for gb in list_of_models:
        list_of_predictions.append(gb.predict(X_from_user))

    top = sorted(list(zip(list_of_predictions, list_of_topics)))[::-1][:3]
    print(top)

In [23]:
with open('texts/economics.txt', encoding='utf-8') as f:
    text = f.read()
    guess_long(text)

with open('texts/culture.txt', encoding='utf-8') as f:
    text = f.read()
    guess_long(text)

with open('texts/internet.txt', encoding='utf-8') as f:
    text = f.read()
    guess_long(text)
    
with open('texts/science.txt', encoding='utf-8') as f:
    text = f.read()
    guess_long(text)

with open('texts/sport.txt', encoding='utf-8') as f:
    text = f.read()
    guess_long(text)

with open('texts/travel.txt', encoding='utf-8') as f:
    text = f.read()
    guess_long(text)

[(array([0.86270357]), 'Экономика'), (array([0.14238702]), 'Дом'), (array([0.11744292]), 'Россия')]
[(array([0.58516109]), 'Культура'), (array([0.13936546]), 'Россия'), (array([0.04576748]), 'Мир')]
[(array([0.50217719]), 'Интернет и СМИ'), (array([0.32055816]), 'Наука и техника'), (array([0.22751611]), 'Экономика')]
[(array([0.54210518]), 'Экономика'), (array([0.50723983]), 'Наука и техника'), (array([0.49693991]), 'Россия')]
[(array([0.5312024]), 'Спорт'), (array([0.41876899]), 'Путешествия'), (array([0.37340028]), 'Интернет и СМИ')]
[(array([0.80754962]), 'Путешествия'), (array([0.13279121]), 'Спорт'), (array([0.06967335]), 'Из жизни')]


Как можно видеть, качество повысилось совсем немного (теперь в самой сложной статье верное предсказание на втором месте). Использование более легкой модели оправдано.